In [ ]:
import holoviews as hv
import hvplot.pandas
import pandas as pd
import xarray as xr
import numpy as np
import os.path as  osp
import glob
import param
hv.extension('bokeh')

### Load TS from a set of ROIs

In [ ]:
DF = pd.DataFrame()
data_path = osp.join('/data','ROI_TS')
roi = 1
for icn in range(1,17):
    ts_files  = glob.glob(data_path+'/*.ICN'+str(icn).zfill(3)+'*.1D')
    for file in ts_files:
        ts = np.loadtxt(file)
        DF['ROI'+str(roi).zfill(3)] = ts
        roi = roi + 1

In [ ]:
TR    = 2
NAcq  = DF.shape[0]
NROIs = DF.shape[1]
time = np.linspace(0, TR*NAcq, NAcq)
DF['Time'] = time
DF.head()

In [ ]:
DF.hvplot(y=['ROI010','ROI020','ROI100'], x='Time').opts(width=1000, legend_position='top_left')

### Use Element (Dynamic Map) to easily explore all time series
A DynamicMap is an explorable multi-dimensional wrapper around a callable that returns HoloViews objects.

In [ ]:
def selected_ts(roi):
    roi = int(roi)
    ROI_ID = 'ROI'+str(roi).zfill(3)
    return DF.hvplot(y=[ROI_ID],x='Time', title=ROI_ID)
dmap = hv.DynamicMap(selected_ts, kdims=['roi'])
dmap.redim.range(roi=(1,NROIs))

### Draw an Interactive Correlation Matrix with HeatMap Object

In [ ]:
CMatrix = DF.drop(['Time'], axis=1).corr()

In [ ]:
CMatrix.head()

In [ ]:
CMatrix.hvplot(kind='heatmap').opts(width=700, height=700, xrotation=45, cmap='RdBu_r', clim=(-1,1))

### Linking Different Elements

In [ ]:
hv_matrix  = CMatrix.hvplot(kind='heatmap').opts(width=500, height=500, xrotation=45, cmap='RdBu_r', clim=(-1,1))
posxy      = hv.streams.Tap(source=hv_matrix, x='ROI001',y='ROI002')
def tap_timeseries(x,y):
    return DF.hvplot(y=[x,y], x='Time', cmap=['red','blue']).opts(width=1000, legend_position='top_left')

In [ ]:
hv_matrix + hv.DynamicMap(tap_timeseries, kdims=[],streams=[posxy])